In [1]:
import requests

# Using Epic Test Server

In [2]:
EPIC_BASE_URL = "https://connectathon.epic.com/interconnect-fhir-oauth/api/FHIR/R4/"

## Scenario 0

In [3]:
DISCOVERY_PATH = ".well-known/smart-configuration"

In [4]:
epic_config = requests.get(EPIC_BASE_URL + DISCOVERY_PATH)

- Payload includes `permission_v2` and `authorize-post` capabilities ✔️
- `code_challenge_methods_supported` includes `S256` ✔️
- `introscpection_endpoint` included ✔️

In [5]:
epic_config.text

'<discovery xmlns="http://hl7.org/fhir"><authorization_endpoint value="https://connectathon.epic.com/Interconnect-Fhir-OAuth/oauth2/authorize" /><token_endpoint value="https://connectathon.epic.com/Interconnect-Fhir-OAuth/oauth2/token" /><token_endpoint_auth_methods_supported value="client_secret_post" /><token_endpoint_auth_methods_supported value="client_secret_basic" /><token_endpoint_auth_methods_supported value="private_key_jwt" /><scopes_supported value="epic.scanning.dmsusername" /><scopes_supported value="fhirUser" /><scopes_supported value="launch" /><scopes_supported value="openid" /><scopes_supported value="profile" /><response_types_supported value="code" /><introspection_endpoint value="https://connectathon.epic.com/Interconnect-Fhir-OAuth/oauth2/introspect" /><capabilities value="launch-ehr" /><capabilities value="launch-standalone" /><capabilities value="client-public" /><capabilities value="client-confidential-symmetric" /><capabilities value="context-banner" /><capabil

## Scenario 1

In [ ]:
authorize_url = "https://connectathon.epic.com/interconnect-fhir-oauth/oauth2/authorize"
token_url = "https://connectathon.epic.com/interconnect-fhir-oauth/oauth2/token"

redirect_uri = "http://localhost:8000/index.html"

client_id = "GranularConnect2021Scenario2"

authorization_redirect_url = "{}?client_id={}&response_type=code&redirect_uri={}".format(authorize_url, 
                                                                                                  client_id, 
                                                                                                  redirect_uri)
    
print("Navigate to: ", authorization_redirect_url + "\n")

authorization_code = input('authz code: ')

headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Accept': 'application/json'}
data = {'grant_type': 'authorization_code', 'code': authorization_code, 'redirect_uri': redirect_uri, 'client_id': client_id}
access_token_response = requests.post(token_url, data=data, headers=headers)

print("\nScopes Granted: " + access_token_response.json()['scope'])

Navigate to:  https://connectathon.epic.com/interconnect-fhir-oauth/oauth2/authorize?client_id=GranularConnect2021Scenario2&response_type=code&redirect_uri=http://localhost:8000/index.html

